<a href="https://colab.research.google.com/github/sandhyaparna/NLP/blob/main/Dell_Take_Home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Import Libraries
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
from google.colab import files
import json
import seaborn as sns
import matplotlib.pyplot as plt
import string, re, os
import collections
from collections import defaultdict, Counter, OrderedDict
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
more_stopwords = ['i']
stop_words = stop_words + more_stopwords
from wordcloud import WordCloud
import textblob
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
print("upload boston_bombing_tweets - Home test Data Scientist.csv file")
tweets_uploaded = files.upload()

upload boston_bombing_tweets - Home test Data Scientist.csv file


Saving boston_bombing_tweets - Home test Data Scientist.csv to boston_bombing_tweets - Home test Data Scientist.csv


In [3]:
print("upload replacements_dict.json file")
dict_uploaded = files.upload()

upload boston_bombing_tweets - Home test Data Scientist.csv file


Saving replacements_dict.json to replacements_dict.json


In [24]:
import io
boston_bombing_tweets = pd.read_csv(io.BytesIO(tweets_uploaded['boston_bombing_tweets - Home test Data Scientist.csv']))
print(f'Tweets data is loaded')

tweets_train = boston_bombing_tweets[pd.notnull(boston_bombing_tweets.label)]
print("Number of observations in train data are", tweets_train.shape[0])

tweets_test = boston_bombing_tweets[pd.isnull(boston_bombing_tweets.label)]
print("Number of observations in test data are", tweets_test.shape[0])

Tweets data is loaded
Number of observations in train data are 4000
Number of observations in test data are 12218


In [7]:
with open('replacements_dict.json') as json_file:
  replacements_dict = json_file.readlines()
  
replacements_dict = map(lambda x: x.rstrip(), replacements_dict)
replacements_dict = ' '.join(replacements_dict)
replacements_dict = json.loads(replacements_dict, object_pairs_hook=OrderedDict)
print("replacements dictionary is loaded")

replacements dictionary is loaded


In [36]:
# helper functions
def replace_all(text, dic=replacements_dict):
    for i, j in dic.items():
        text = re.sub(i, j, text)
    return text

def text_preprocess(text):
  text = re.sub('[%s]' % re.escape(string.punctuation.replace('#','')), '', text)
  text = re.sub('^# |# ', '', text)
  text = re.sub('[^\u0000-\u007e]+', '', text)
  text = ' '.join(word for word in text.split(' ') if word not in stop_words)
  return text

# def create_corpus(data, text_var, target_var, target_value):
#     corpus=[]
#     for x in data[data[target_var]==target_value][text_var].str.split():
#         for i in x:
#             corpus.append(i)
#     return corpus

def create_corpus(data, text_var, condition):
    corpus=[]
    
    if condition == "hash":
      for x in data['text_processed'].apply(lambda x: re.findall(r'(?<=)#\w+',x)):
          for i in x:
            corpus.append(i)
    
    else:  # non-hash words
      for x in data['text_processed'].apply(lambda x: re.findall(r'(?#)\w+',x)):
        for i in x:
          corpus.append(i)

    return corpus

def get_top_tweet_ngrams(corpus, n):
    vec = CountVectorizer(ngram_range=(n, n), token_pattern=r'\b\w\w+\b|(?<!\w)#\w+').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:None]

In [30]:
def preprocessing_fn(data, text_var):
  """
  delete missing values, 

  """
  data = data[pd.notnull(data[text_var])]
  data[text_var+'_processed'] = data[text_var].str.lower()
  data[text_var+'_URL_present'] = np.where(data[text_var+'_processed'].str.contains("http://"),1,0)
  data[text_var+'_retweet'] = np.where(data[text_var+'_processed'].str.contains("rt @"),1, 0)

  data[text_var+'_processed'] = data[text_var+'_processed'].apply(lambda x: replace_all(x,replacements_dict))
  data[text_var+'_punctuations_count'] = data[text_var+'_processed'].apply(lambda x: re.findall('[%s]' % re.escape(string.punctuation),str(x))).str.len() 
  data[text_var+'_word_count'] = data[text_var+'_processed'].apply(lambda x: len(str(x).split()))
  data[text_var+'_hashtag_count'] = data[text_var+'_processed'].apply(lambda x: len([c for c in str(x) if c == '#']))
  
  data[text_var+'_processed'] = data[text_var+'_processed'].apply(lambda x: text_preprocess(x))

  return data

In [31]:
tweets_train_x = preprocessing_fn(tweets_train, "text")
tweets_train_x.head()

,text,label,text_processed,text_URL_present,text_retweet,text_punctuations_count,text_word_count,text_hashtag_count
1,RT @wilw NBC reporting suspect alive and in cu...,1.0,nbc reporting suspect alive custody cnn repor...,0,1,7,21,0
2,#BostonMararthon suspect Dzhokhar Tsarnaev is ...,1.0,#bostonmararthon suspect dzhokhar tsarnaev cus...,1,0,1,7,1
6,RT @BreitbartNews Margaret Thatcher Remembered...,1.0,margaret thatcher remembered america midst te...,1,1,4,17,0
15,Spontaneous celebrations in streets of #Watert...,1.0,spontaneous celebrations streets #watertown #b...,0,0,6,24,2
25,#Bostonsgigs Looking for someone for dishwashe...,0.0,#bostonsgigs looking someone dishwasher instal...,1,0,5,10,2


In [ ]:
def feature_engineering(data):
  features = []

  for i, j in Counter(create_corpus(data, "text_processed", "non-hash")).items():
    if j > 30:
      features.append(i)

  for i, j in Counter(create_corpus(data, "text_processed", "hash")).items():
    if j > 10:
      features.append(i)

  for i, j in get_top_tweet_ngrams(data["text_processed"], 2):
    if j > 10:
      features.append(i)

  for i, j in get_top_tweet_ngrams(data["text_processed"], 3):
    if j > 5:
      features.append(i)

  print(features)
  print(len(features))

  for i in features:
    data[i+"_word"] = np.where(data["text_processed"].str.contains(i),1,0)
  
  data.columns = data.columns.str.replace(' ', '_')
  
  return data

tweets_train_y = feature_engineering(tweets_train_x)
tweets_train_y.head()

In [ ]:
print(*list(tweets_train_y))

In [ ]:
for i in features[250:]:
  tweets_train[i] = np.where(tweets_train["text_processed"].str.contains(i),1,0)

In [ ]:
features[290:]
tweets_train['boston boston'] = np.where(tweets_train["text_processed"].str.contains('boston boston'),1,0)
tweets_train.columns = tweets_train.columns.str.replace(' ', '_')

In [ ]:
i.replace(" ","_")

In [ ]:
tweets_train.head()